# Import Libraries

In [1]:
import numpy as np
import pandas as pd
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader, Subset

from sklearn.model_selection import GroupKFold

In [2]:
num_kfolds = 5
batch_size = 32
learning_rate = 3e-3
num_epoch = 10
es_patience = 20
quantiles = (0.2, 0.5, 0.8)
model_name = 'descartes'

# Create Metric Function

In [3]:
def laplace_log_likelihood(actual_FVC, predicted_FVC, confidence, return_values = False):
    sd_clipped = np.maximum(confidence, 70)
    delta = np.minimum(np.abs(actual_FVC - predicted_FVC), 1000)
    metric = -np.sqrt(2) * delta / sd_clipped - np.log(np.sqrt(2) * sd_clipped)
    
    if return_values:
        return metric
    else:
        return np.mean(metric)

# Create Dataset

In [4]:
class MedicalDataset(Dataset):
    def __init__(self, mode, transform=None):
        self.transform = transform
        self.mode = mode
        
        train = pd.read_csv('../input/osic-pulmonary-fibrosis-progression/train.csv')
        
        # Some of the patients have inspected twice a week. So, remove duplicated records. 
        train.drop_duplicates(keep=False, inplace=True, subset=['Patient', 'Weeks'])
        
        sub = pd.read_csv('../input/osic-pulmonary-fibrosis-progression/sample_submission.csv')
        sub['Patient'] = sub['Patient_Week'].apply(lambda x: x.split('_')[0])
        sub['Weeks'] = sub['Patient_Week'].apply(lambda x: int(x.split('_')[-1]))
        sub = sub[['Patient', 'Weeks', 'Confidence', 'Patient_Week']]
        
        chunk = pd.read_csv('../input/osic-pulmonary-fibrosis-progression/test.csv')
        sub = sub.merge(chunk.drop('Weeks', axis=1), on='Patient')
        
        train['WHERE'] = 'train'
        chunk['WHERE'] = 'val'
        sub['WHERE'] = 'test'
        
        # Merge train, test, test data.
        data = train.append([chunk, sub])
        
        # Calculate min_week by each patient.
        data['min_week'] = data['Weeks']
        data.loc[data.WHERE == 'test', 'min_week'] = np.nan
        data['min_week'] = data.groupby('Patient')['min_week'].transform('min')
        
        init_exam = data.loc[data.Weeks == data.min_week]
        init_exam = init_exam[['Patient', 'FVC']].copy()
        init_exam.columns = ['Patient', 'min_FVC']
        init_exam['nb'] = 1
        init_exam['nb'] = init_exam.groupby('Patient')['nb'].transform('cumsum')
        init_exam = init_exam[init_exam.nb == 1]
        init_exam.drop('nb', axis=1, inplace=True)
        
        # Calculate day
        data = data.merge(init_exam, on='Patient', how='left')
        data['base_week'] = data['Weeks'] - data['min_week']
        del init_exam
        
        # Transform 'Sex' and 'SmokingStatus' data into One-Hot vector.
        COLS = ['Sex', 'SmokingStatus']
        self.FE = []
        for col in COLS:
            for mod in data[col].unique():
                self.FE.append(mod)
                data[mod] = (data[col] == mod).astype(int)
        
        # Normalize each data so that the maximum value is 1 and the minimum value is 0.
        data['age'] = (data['Age'] - data['Age'].min()) / \
                      (data['Age'].max() - data['Age'].min())
        
        data['BASE'] = (data['min_FVC'] - data['min_FVC'].min()) / \
                      (data['min_FVC'].max() - data['min_FVC'].min())

        data['week'] = (data['base_week'] - data['base_week'].min()) / \
                      (data['base_week'].max() - data['base_week'].min())

        data['percent'] = (data['Percent'] - data['Percent'].min()) / \
                      (data['Percent'].max() - data['Percent'].min())
        
        self.FE += ['age', 'percent', 'week', 'BASE']
        self.raw = data.loc[data.WHERE == mode].reset_index()
        del data
        
        
    def __len__(self):
        return len(self.raw)
    
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
            
        sample = {
            'patient_id': self.raw['Patient'].iloc[idx],
            'features': self.raw[self.FE].iloc[idx].values,
            'target': self.raw['FVC'].iloc[idx]
        }
        if self.transform:
            sample = transform(sample)
    
        return sample
    
    
    def group_kfold(self, n_splits):
        gkf = GroupKFold(n_splits=n_splits)
        groups = self.raw['Patient']
        for train_idx, val_idx in gkf.split(self.raw, self.raw, groups):
            train = Subset(self, train_idx)
            val = Subset(self, val_idx)
            yield train, val
        

# Create Model (Quantile Regression Neural Network)

In [5]:
class QrnnModel(nn.Module):
    def __init__(self, in_tabular_features=9, out_quantiles=3):
        super(QrnnModel, self).__init__()
        self.fc1 = nn.Linear(in_tabular_features, 100)
        self.bn1 = nn.BatchNorm1d(num_features=100)
        self.fc2 = nn.Linear(100, 100)
        self.fc3 = nn.Linear(100, out_quantiles)
    
    def forward(self, x):
        x = F.leaky_relu(self.fc1(x))
        x = self.bn1(x)
        x = F.leaky_relu(self.fc2(x))
        x = self.fc3(x)
        
        return x
    

def quantile_loss(preds, target, quantiles):
    losses = []
    for i, q in enumerate(quantiles):
        errors = target - preds[:, i]
        losses.append(torch.max((q - 1) * errors, q * errors).unsqueeze(1))
    loss = torch.mean(torch.sum(torch.cat(losses, dim=1), dim=1))
    return loss

# Train Model

In [54]:
models = []
data = MedicalDataset(mode='train')
folds = data.group_kfold(num_kfolds)

MAX = 10000000
best_metric_score = [-10000] * num_kfolds
for fold, (trainset, valset) in enumerate(folds):
    print("***************** Fold No.{} ********************".format([fold]))
    
    dataloaders = {
        'train': DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2),
        'val': DataLoader(valset, batch_size=batch_size, shuffle=False, num_workers=2)
    }
    
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = QrnnModel().to(device)
    optimizer = Adam(model.parameters(), lr=learning_rate)
    
    for epoch in range(num_epoch):
        
        all_pred = []
        all_target = []
        all_loss = []
        all_metrix_loss = 0
        
        # train
        itr = 1
        for batch in dataloaders['train']:
            model.train()
            inputs = batch['features'].float().to(device)
            targets = batch['target'].to(device)

            optimizer.zero_grad()

            with torch.set_grad_enabled(True):
                preds = model(inputs)
                loss = quantile_loss(preds, targets, quantiles)
                loss.backward()
                optimizer.step()
                all_loss.append(loss)

            if itr % 10 == 0:
                print(f"Epoch #{epoch+1}, Iteration #{itr}, loss: {loss}")
            itr += 1

        # validation
        for batch in dataloaders['val']:
            model.eval()
            inputs = batch['features'].float().to(device)
            target = batch['target']
            optimizer.zero_grad()
            with torch.set_grad_enabled(False):
                preds = model(inputs)
                all_pred.extend(preds.detach().cpu().numpy().tolist())
                all_target.extend(target.numpy().tolist())
        
        all_pred = torch.FloatTensor(all_pred)
        all_target = torch.FloatTensor(all_target)
        all_target=torch.reshape(all_target, all_pred[:,1].shape)
        all_confidence = all_pred[:, 2] - all_pred[:, 0]
        metric_loss = laplace_log_likelihood(all_pred[:, 1], all_target, all_confidence, True)

        print(f"Epoch #{epoch+1}, Metric loss: {metric_loss.mean()}\n")
        best_metric_score[fold] = max(best_metric_score[fold], metric_loss.mean())
            
    models.append(model)

for fold in range(num_kfolds):
    print(f"best score in fold {fold}: {best_metric_score[fold]}")

***************** Fold No.[0] ********************
Epoch #1, Iteration #10, loss: 4146.05859375
Epoch #1, Iteration #20, loss: 4119.82470703125
Epoch #1, Iteration #30, loss: 4186.79736328125
Epoch #1, Metric loss: -24.798133850097656

Epoch #2, Iteration #10, loss: 3734.156494140625
Epoch #2, Iteration #20, loss: 3730.788818359375
Epoch #2, Iteration #30, loss: 3679.80908203125
Epoch #2, Metric loss: -24.65211296081543

Epoch #3, Iteration #10, loss: 3177.709716796875
Epoch #3, Iteration #20, loss: 2385.972900390625
Epoch #3, Iteration #30, loss: 1786.61279296875
Epoch #3, Metric loss: -15.424593925476074

Epoch #4, Iteration #10, loss: 1748.4720458984375
Epoch #4, Iteration #20, loss: 513.23388671875
Epoch #4, Iteration #30, loss: 317.7638244628906
Epoch #4, Metric loss: -7.435503959655762

Epoch #5, Iteration #10, loss: 782.9888916015625
Epoch #5, Iteration #20, loss: 554.678466796875
Epoch #5, Iteration #30, loss: 349.2149963378906
Epoch #5, Metric loss: -7.400668144226074

Epoch #

# Predict Testdata

In [46]:
data = MedicalDataset(mode='test')
avg_preds = np.zeros((len(data), len(quantiles)))
                     
for model in models:
    dataloader = DataLoader(data, batch_size=batch_size, shuffle=False, num_workers=2)
    
    preds = []
    for batch in dataloader:
        inputs = batch['features'].float()
        with torch.no_grad():
            x = model(inputs)
            preds.append(x)
            
    preds = torch.cat(preds, dim=0).numpy()
    avg_preds += preds
    
avg_preds /= len(models)
df = pd.DataFrame(data = avg_preds, columns=list(quantiles))
df['Patient_Week'] = data.raw['Patient_Week']
df['FVC'] = df[quantiles[1]]
df['Confidence'] = df[quantiles[2]] - df[quantiles[0]]
df = df.drop(columns=list(quantiles))
df.to_csv('submission.csv', index=False)

In [44]:
df.head()

,Patient_Week,FVC,Confidence
0,ID00419637202311204720264_-12,2842.916162,771.930811
1,ID00419637202311204720264_-11,2840.654639,771.317822
2,ID00419637202311204720264_-10,2838.393066,770.704639
3,ID00419637202311204720264_-9,2836.131201,770.091162
4,ID00419637202311204720264_-8,2833.869775,769.478174
